# Strands Agents와 AgentCore Memory (단기 메모리)


## 소개

이 튜토리얼은 AgentCore **단기 메모리**(Raw events)를 사용하여 Strands agents로 **개인 에이전트**를 구축하는 방법을 보여줍니다. 에이전트는 `get_last_k_turns`를 사용하여 세션의 최근 대화를 기억하고 사용자가 돌아왔을 때 대화를 원활하게 계속할 수 있습니다.


### 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 단기 대화형                                                                        |
| 에이전트 유형        | 개인 에이전트                                                                      |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                      |
| 튜토리얼 구성요소     | AgentCore 단기 메모리, AgentInitializedEvent 및 MessageAddedEvent 훅             |
| 예제 복잡도          | 초급                                                                             |

배울 내용:
- 대화 연속성을 위한 단기 메모리 사용
- 마지막 K개 대화 턴 검색
- 실시간 정보를 위한 웹 검색 도구
- 대화 기록으로 에이전트 초기화

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 전제조건

- Python 3.10+
- AgentCore Memory 권한이 있는 AWS 자격 증명
- AgentCore Memory 역할 ARN
- Amazon Bedrock 모델에 대한 액세스

환경 설정을 시작해보겠습니다!

## 1단계: 설정 및 가져오기

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

# Setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [ ]:
# Imports
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# Configuration
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
ACTOR_ID = "user_123" # It can be any unique identifier (AgentID, User ID, etc.)
SESSION_ID = "personal_session_001" # Unique session identifier


## 2단계: 웹 검색 도구

먼저 에이전트를 위한 간단한 웹 검색 도구를 만들어보겠습니다.

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

## 3단계: 메모리 리소스 생성
단기 메모리의 경우, 전략 없이 메모리 리소스를 생성합니다. 이는 `get_last_k_turns`로 검색할 수 있는 원시 대화 턴을 저장합니다.


In [ ]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # Create memory resource without strategies (thus only access to short-term memory)
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for personal agent",
        event_expiry_days=7, # Retention period for short-term memory. This can be upto 365 days.
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")

## 4단계: 메모리 훅

이 단계에서는 메모리 작업을 자동화하는 사용자 정의 `MemoryHookProvider` 클래스를 정의합니다. 훅은 에이전트의 실행 생명주기의 특정 지점에서 실행되는 특수 함수입니다. 우리가 만드는 메모리 훅은 두 가지 주요 기능을 제공합니다:
1. **최근 대화 로드**: `AgentInitializedEvent` 훅을 사용하여 에이전트가 초기화될 때 최근 대화 기록을 자동으로 로드합니다.
2. **마지막 메시지 저장**: 새로운 대화 메시지를 저장합니다.

이는 수동 관리 없이 원활한 메모리 경험을 만들어줍니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Load the last 5 conversation turns from memory
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt.
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory"""
        messages = event.agent.messages
        try:
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(str(messages[-1].get("content", "")), messages[-1]["role"])]
            )
        except Exception as e:
            logger.error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## 5단계: 웹 검색 기능이 있는 개인 에이전트 생성

In [ ]:
def create_personal_agent():
    """Create personal agent with memory and web search"""
    agent = Agent(
        name="PersonalAssistant",
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # or your preferred model
        system_prompt=f"""You are a helpful personal assistant with web search capabilities.
        
        You can help with:
        - General questions and information lookup
        - Web searches for current information
        - Personal task management
        
        When you need current information, use the websearch function.
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly and professional.""",
        hooks=[MemoryHookProvider(client, memory_id, ACTOR_ID, SESSION_ID)],
        tools=[websearch],
    )
    return agent

# Create agent
agent = create_personal_agent()
logger.info("✅ Personal agent created with memory and web search")

#### 축하합니다! 에이전트가 준비되었습니다! :) 
## 에이전트를 테스트해보겠습니다

In [ ]:
# Test conversation with memory
print("=== First Conversation ===")
print(f"User: My name is Alex and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

In [ ]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

In [ ]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

## 메모리 연속성 테스트

메모리 시스템이 올바르게 작동하는지 테스트하기 위해 에이전트의 새 인스턴스를 생성하고 이전에 저장된 정보에 액세스할 수 있는지 확인해보겠습니다:

In [ ]:
# Create new agent instance (simulates user returning)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()

# Test memory continuity
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

## 저장된 메모리 보기

In [ ]:
# Check what's stored in memory
print("=== Memory Contents ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # Adjust k to see more or fewer turns
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## 요약

이 튜토리얼에서는 개인 에이전트를 구축하는 방법을 보여주었습니다. 배운 내용:

- 전략 없이 메모리 리소스 생성
- 대화 기록을 위한 `get_last_k_turns` 사용
- 에이전트에 웹 검색 기능 추가
- 컨텍스트 로딩을 위한 메모리 훅 구현

**다음 단계:**
- 더 정교한 도구 추가
- 장기 메모리 전략 구현
- 여러 소스로 검색 기능 향상

## 정리 (선택사항)

In [ ]:
# Uncomment to delete memory resource
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ Deleted memory: {memory_id}")